# Plotting TRIOS Complex Modulus Data

This notebook demonstrates the **correct** way to plot complex modulus data loaded from TRIOS files.

## The Problem

TRIOS frequency sweep data contains both Storage (G') and Loss (G'') moduli. RheoJAX correctly loads this as complex modulus:
```python
G* = G' + i·G''
```

However, matplotlib **cannot plot complex-valued arrays directly**. This causes warnings:
- `ComplexWarning: Casting complex values to real discards the imaginary part`
- `UserWarning: Data has no positive values, and therefore cannot be log-scaled`

## Solutions

We provide **three correct approaches** below.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from rheojax.io import load_trios

# Load TRIOS data (returns complex modulus)
data_list = load_trios('../../data/experimental/frequency_sweep_tts.txt')
print(f"Loaded {len(data_list)} temperature segments")
print(f"Data type: {data_list[0].y.dtype}")
print(f"Is complex: {np.iscomplexobj(data_list[0].y)}")

## ❌ WRONG: Direct Plotting (Causes Warnings)

**DO NOT DO THIS:**

In [ ]:
# ❌ WRONG - This will trigger ComplexWarning
# plt.loglog(data_list[0].x, data_list[0].y)
# plt.tight_layout()  # <-- Warnings appear here

print("⚠️  Commented out to prevent warnings. This is the INCORRECT approach.")

## ✅ Solution 1: Use RheoJAX Visualization (Recommended)

The easiest and most reliable approach is to use RheoJAX's built-in plotting functions that automatically handle complex modulus data.

In [ ]:
from rheojax.visualization import plot_rheo_data

# ✅ CORRECT - Automatic complex modulus handling
fig, axes = plot_rheo_data(data_list[0])
plt.suptitle(f"Temperature: {data_list[0].metadata.get('temperature', 'N/A')}°C", y=1.02)
plt.tight_layout()
plt.show()

print("✓ No warnings! G' and G'' plotted in separate subplots.")

## ✅ Solution 2: Manual Component Extraction

If you need custom matplotlib control, manually extract the real and imaginary components.

In [ ]:
# ✅ CORRECT - Manual extraction of G' and G''
data = data_list[0]

# Extract components from complex modulus
omega = data.x  # Frequency (rad/s)
G_prime = np.real(data.y)  # Storage modulus (G')
G_double_prime = np.imag(data.y)  # Loss modulus (G'')

# Plot on separate axes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

# G' (Storage modulus)
ax1.loglog(omega, G_prime, 'o-', label="G' (Storage)", color='C0')
ax1.set_xlabel('Frequency (rad/s)')
ax1.set_ylabel("G' (Pa)")
ax1.legend()
ax1.grid(True, alpha=0.3)

# G'' (Loss modulus)
ax2.loglog(omega, G_double_prime, 's-', label='G" (Loss)', color='C1')
ax2.set_xlabel('Frequency (rad/s)')
ax2.set_ylabel('G" (Pa)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle(f"Temperature: {data.metadata.get('temperature', 'N/A')}°C")
plt.tight_layout()
plt.show()

print("✓ No warnings! Components extracted before plotting.")

## ✅ Solution 3: Plot G' and G'' on Same Axes

For comparison, plot both components on a single log-log plot.

In [ ]:
# ✅ CORRECT - Both components on same axes
data = data_list[0]
omega = data.x
G_prime = np.real(data.y)
G_double_prime = np.imag(data.y)

fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(omega, G_prime, 'o-', label="G' (Storage)", color='C0', markersize=5)
ax.loglog(omega, G_double_prime, 's-', label='G" (Loss)', color='C1', markersize=5)

ax.set_xlabel('Frequency (rad/s)', fontsize=12)
ax.set_ylabel('Modulus (Pa)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, which='both')
ax.set_title(f"Complex Modulus - Temperature: {data.metadata.get('temperature', 'N/A')}°C", fontsize=13)

plt.tight_layout()
plt.show()

print("✓ No warnings! Both components on same axes.")

## Plot All Temperature Segments

Demonstrate plotting multiple temperature segments correctly.

In [ ]:
# ✅ CORRECT - Plot all 20 temperature segments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Use a colormap for temperature gradient
cmap = plt.cm.coolwarm
n_segments = len(data_list)

for i, data in enumerate(data_list):
    omega = data.x
    G_prime = np.real(data.y)
    G_double_prime = np.imag(data.y)
    
    temp = data.metadata.get('temperature', 0)
    color = cmap(i / n_segments)
    
    # Plot G'
    ax1.loglog(omega, G_prime, 'o-', color=color, alpha=0.7, markersize=3,
               label=f"{temp}°C" if i % 4 == 0 else None)  # Label every 4th for clarity
    
    # Plot G''
    ax2.loglog(omega, G_double_prime, 's-', color=color, alpha=0.7, markersize=3,
               label=f"{temp}°C" if i % 4 == 0 else None)

ax1.set_xlabel('Frequency (rad/s)', fontsize=12)
ax1.set_ylabel("G' (Pa)", fontsize=12)
ax1.set_title("Storage Modulus (G')", fontsize=13)
ax1.legend(title='Temperature', fontsize=9)
ax1.grid(True, alpha=0.3, which='both')

ax2.set_xlabel('Frequency (rad/s)', fontsize=12)
ax2.set_ylabel('G" (Pa)', fontsize=12)
ax2.set_title('Loss Modulus (G")', fontsize=13)
ax2.legend(title='Temperature', fontsize=9)
ax2.grid(True, alpha=0.3, which='both')

plt.suptitle('TRIOS Frequency Sweep - All Temperature Segments', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(f"✓ Successfully plotted {n_segments} temperature segments without warnings!")

## Verify Data Quality

Check that all values are positive (required for log-scale).

In [ ]:
# Check data quality across all segments
all_G_prime = np.concatenate([np.real(d.y) for d in data_list])
all_G_double_prime = np.concatenate([np.imag(d.y) for d in data_list])

print("Data Quality Report:")
print("=" * 50)
print(f"G' (Storage Modulus):")
print(f"  Range: {all_G_prime.min():.1f} - {all_G_prime.max():.1f} Pa")
print(f"  All positive: {np.all(all_G_prime > 0)}")
print(f"\nG'' (Loss Modulus):")
print(f"  Range: {all_G_double_prime.min():.1f} - {all_G_double_prime.max():.1f} Pa")
print(f"  All positive: {np.all(all_G_double_prime > 0)}")
print(f"\nLoss Tangent (tan δ = G''/G'):")
tan_delta = all_G_double_prime / all_G_prime
print(f"  Range: {tan_delta.min():.3f} - {tan_delta.max():.3f}")
print(f"  Material type: {'Elastic-dominant' if tan_delta.mean() < 1 else 'Viscous-dominant'}")
print("=" * 50)

if np.all(all_G_prime > 0) and np.all(all_G_double_prime > 0):
    print("\n✓ All data values are positive - safe for log-scale plotting!")
else:
    print("\n⚠️  Warning: Some values are non-positive")

## Summary

### Key Takeaways

1. **TRIOS data is complex-valued**: `G* = G' + i·G''`
2. **Matplotlib cannot plot complex arrays**: Must extract real/imaginary parts first
3. **Three correct approaches**:
   - Use `plot_rheo_data()` (easiest)
   - Manual extraction with `np.real()` and `np.imag()`
   - Custom matplotlib plotting after component extraction

### What NOT to Do

```python
# ❌ WRONG - Triggers ComplexWarning
plt.loglog(data.x, data.y)
```

### What TO Do

```python
# ✅ CORRECT - Option 1 (Recommended)
from rheojax.visualization import plot_rheo_data
plot_rheo_data(data)

# ✅ CORRECT - Option 2 (Custom plotting)
plt.loglog(data.x, np.real(data.y), label="G'")
plt.loglog(data.x, np.imag(data.y), label='G"')
```